## Simple CNN network

The original problem is:

    (P) Compute the fabrics from multiple sliced images per rev

We are firstly having an intermediate objective:

    (P') Compute the fabrics from one single sliced images per rev

Of course, the accuracy will be much worse because the fabrics are computed in every direction. However, it is a good try.

# Importing the dataframe

Firstly, we initialize wandb. It is a tool that allows to store the losses and retrieve the deframe. Otherwise, you can directly access locally the dataframe on your computer.

In [ ]:
!pip install wandb --upgrade

We import all the useful packages.

In [ ]:
import sys
from pathlib import Path

IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules
if IS_KAGGLE:
    repo_path = Path("../input/microstructure-reconstruction")
elif IS_COLAB:
    from google.colab import drive

    drive.mount("/content/gdrive")
    repo_path = Path("/content/gdrive/MyDrive/microstructure-reconstruction")
else:
    repo_path = Path("/home/matias/microstructure-reconstruction")
sys.path.append(str(repo_path))

from copy import deepcopy
from importlib import reload

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pandas as pd
import pytorch_lightning as pl
import torch
from typing import Union, List
import torch.nn as nn
import torch.optim as optim
import torchmetrics
import torchvision.models as pretrained_models
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KernelDensity
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader
from torchvision import transforms, utils
from tqdm import tqdm

import wandb
from custom_datasets import dataset
from custom_models import models
from tools import dataframe_reformat, inspect_code, plotting, training, wandb_api

log_wandb = True

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 2, "pin_memory": True} if use_cuda else {"num_workers": 4}
print(f"[INFO]: Computation device: {device}")


We initialize a wandb run, that will save our metrics

In [ ]:
if log_wandb:
    import wandb
    wandb_api.login()
    run = wandb.init(
        project="microstructure-reconstruction",
        group="Naive Network",
        job_type="test",
    )


Parameters of our run:

In [ ]:
if log_wandb:
    config = wandb.config
else:
    config = wandb_api.Config()

config["job_type"] = run.job_type
config["train_val_split"] = 0.7
config["seed"] = 42
config["batch_size"] = 32
config["learning_rate"] = 0.0003
config["device"] = device
config["momentum"] = 0.7
config["architecture"] = "VGG"
config["input_width"] = 100
config["weight_decay"] = 0.00005
config["epochs"] = 0
config["frac_sample"] = 1
config["frac_noise"] = 0.9
config["log_wandb"] = log_wandb
torch.manual_seed(config["seed"])
pl.seed_everything(config["seed"])

In [ ]:
class DataModule(pl.LightningDataModule):
    def __init__(
        self,
        config,
        repo_path,
        train_df=None,
        test_df=None,
    ):
        super().__init__()
        self.config = config
        self.repo_path = repo_path
        self.train_df = train_df.convert_dtypes() if train_df is not None else None
        self.test_df = test_df.convert_dtypes() if test_df is not None else None

        if self.config["log_wandb"]:
            if self.train_df is None:
                self.training_data_at = wandb.Api().artifact(
                    "matiasetcheverry/microstructure-reconstruction/train_df:10_images"
                )
            if self.test_df is None:
                self.test_data_at = wandb.Api().artifact(
                    "matiasetcheverry/microstructure-reconstruction/test_df:10_images"
                )

        self.transform = transforms.Compose(
            [
                transforms.CenterCrop(207),
                transforms.Resize(
                    (self.config["input_width"], self.config["input_width"])
                ),
                transforms.ToTensor(),
                transforms.GaussianBlur(kernel_size=3, sigma=0.5),
            ]
        )

    def prepare_data(self):
        if self.config["log_wandb"]:
            if self.train_df is None:
                self.training_data_at.download()
            if self.test_df is None:
                self.test_data_at.download()

    def _init_df_wandb(self):
        if self.train_df is None:
            self.train_df = wandb_api.convert_table_to_dataframe(
                self.training_data_at.get("fabrics")
            )
            self.train_df["photos"] = self.train_df["photos"].apply(
                func=lambda photo_paths: [
                    str(self.repo_path / Path(x)) for x in photo_paths
                ]
            )
        if self.test_df is None:
            self.test_df = wandb_api.convert_table_to_dataframe(
                self.test_data_at.get("fabrics")
            )
            self.test_df["photos"] = self.test_df["photos"].apply(
                func=lambda photo_paths: [
                    str(self.repo_path / Path(x)) for x in photo_paths
                ]
            )

    def _init_df_local(self):
        fabrics_df = pd.read_csv(self.repo_path / "REV1_600/fabrics.txt")
        path_to_slices = self.repo_path / "REV1_600/REV1_600Slices"
        fabrics_df["photos"] = fabrics_df["id"].apply(
            func=dataframe_reformat.associate_rev_id_to_its_images,
            args=(path_to_slices, 10),
        )
        fabrics_df = fabrics_df[fabrics_df.photos.str.len().gt(0)]
        fabrics_df["photos"] = fabrics_df["photos"].apply(func=lambda x: sorted(x))
        train_df, test_df = train_test_split(
            fabrics_df,
            train_size=config["train_val_split"],
            random_state=config["seed"],
            shuffle=True,
        )
        if self.train_df is None:
            self.train_df = train_df.reset_index(drop=True)
        if self.test_df is None:
            self.test_df = test_df.reset_index(drop=True)

    def init_df(self):
        if self.config["log_wandb"]:
            self._init_df_wandb()
        else:
            self._init_df_local()

    def setup(self, stage):
        self.init_df()
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.scaler.fit_transform(self.train_df.iloc[:, 1:-1])
        self.scaler.transform(self.test_df.iloc[:, 1:-1])
        normalized_train_df = deepcopy(self.train_df)
        normalized_train_df.iloc[:, 1:-1] = self.scaler.transform(
            self.train_df.iloc[:, 1:-1]
        )
        normalized_test_df = deepcopy(self.test_df)
        normalized_test_df.iloc[:, 1:-1] = self.scaler.transform(
            self.test_df.iloc[:, 1:-1]
        )
        self.kde = KernelDensity(kernel="gaussian", bandwidth=0.75).fit(
            normalized_train_df.iloc[:, 1:-1].to_numpy()
        )

        self.train_dataset = dataset.SinglePhotoDataset(
            normalized_train_df, transform=self.transform, noise=config["frac_noise"]
        )
        self.validation_dataset = dataset.SinglePhotoDataset(
            normalized_test_df, transform=self.transform, noise=0
        )
        self.targets = self.test_df.iloc[:, 1:-1].to_numpy()

        

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.config["batch_size"],
            shuffle=True,
            **kwargs,
        )

    def val_dataloader(self):
        return DataLoader(
            self.validation_dataset,
            batch_size=self.config["batch_size"],
            shuffle=False,
            **kwargs,
        )

    def test_dataloader(self):
        return self.val_dataloader()

    def predict_dataloader(self):
        return DataLoader(
            [image for image, _ in self.validation_dataset],
            batch_size=self.config["batch_size"],
            shuffle=False,
            **kwargs,
        )


dm = DataModule(config, repo_path)


In [ ]:
dm.prepare_data()
dm.setup(stage="fit")
first_batch = next(iter(dm.train_dataloader()))

images, labels = first_batch[0], first_batch[1]
grid = utils.make_grid(images)
fig = plt.figure(figsize=(40, 10))
plt.imshow(grid.numpy().transpose((1, 2, 0)))
plt.show()

In [ ]:
class VGG11(models.BaseModel):
    def __init__(self, config, kde = None, scaler=None):
        super().__init__(config)

        self.config = config
        self.config["model_type"] = type(self)
        self.kde = kde
        self.scaler = scaler

        self.configure_model()
        self.configure_criterion()
        self.configure_metrics()
        
    def configure_model(self):
        layers = np.array([64, 128, 256, 512]) // 4
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, layers[0], kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(layers[0], layers[1], kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(layers[1], layers[2], kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(layers[2], layers[2], kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(layers[2], layers[3], kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(layers[3], layers[3], kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(layers[3], layers[3], kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(layers[3], layers[3], kernel_size=3, padding=1),
            nn.BatchNorm2d(layers[3]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
#             nn.MaxPool2d(kernel_size=int(self.config["input_width"] / (2 ** 4)), stride=int(self.config["input_width"] / (2 ** 4))),
        )
        input_fc = int((self.config["input_width"] / (2 ** 5)) ** 2 * layers[3])
        # fully connected linear layers
        n = 256
        self.linear_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_fc, out_features=n),
            nn.ReLU(),
            nn.Dropout2d(0.3),
            nn.Linear(in_features=n, out_features=1),
            nn.ReLU(),
            nn.Dropout2d(0.3),
            nn.Linear(in_features=n, out_features=1),
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        # flatten to prepare for the fully connected layers
#         x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
#         weights = self.kde.score_samples(y_hat.cpu().detach().numpy())
        loss = self.criterion(y_hat, y)
        self.log(
            "train_loss",
            loss,
            on_step=False,
            on_epoch=True,
        )
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
#         weights = self.kde.score_samples(y_hat.cpu().detach().numpy())
        metrics = {name: metric(y, y_hat) for name, metric in self.metrics.items()}
        metrics["val_loss"] = self.criterion(y_hat, y)
        self.log_dict(metrics, on_step=False, on_epoch=True, prog_bar=True)
        return metrics

    def configure_criterion(self):
        self.criterion = self.L1Loss
        self.config["loss_type"] = type(self.criterion)

    def L1Loss(self, y, y_hat, weights=None):
        if weights is not None:
            weights = nn.functional.normalize(weights, p=1, dim=0)
            return torch.mean(torch.matmul(weights, torch.abs(y - y_hat)))
        else: 
            return torch.mean(torch.abs(y - y_hat))

    def configure_metrics(self):
        self.metrics = {
            "mae": torchmetrics.MeanAbsoluteError().to(self.config["device"]),
            "mape": torchmetrics.MeanAbsolutePercentageError().to(
                self.config["device"]
            ),
            "smape": torchmetrics.SymmetricMeanAbsolutePercentageError().to(
                self.config["device"]
            ),
            "r2_score": torchmetrics.R2Score(num_outputs=23).to(self.config["device"]),
            "cosine_similarity": torchmetrics.CosineSimilarity(reduction="mean").to(
                self.config["device"]
            ),
        }
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(),
            lr=self.config["learning_rate"],
            weight_decay=self.config["weight_decay"],
        )
        self.config["optimizer_type"] = type(optimizer)
        return optimizer


model = VGG11(config, kde=dm.kde)
total_params = sum(p.numel() for p in model.parameters())
print(f"[INFO]: {total_params:,} total parameters.")
model(torch.rand((2, 1, config["input_width"], config["input_width"])))


In [ ]:
model_checkpoint = pl.callbacks.model_checkpoint.ModelCheckpoint(
    dirpath=run.dir,
    filename="{epoch}-{val_loss:.3f}",
    monitor="val_loss",
    mode="min",
    verbose=True,
    save_last=True,
)

script_checkpoint = training.ScriptCheckpoint(
    dirpath=run.dir,
)

callbacks = [script_checkpoint]
log = None
if run.job_type == "train" or True:
    callbacks.append(model_checkpoint)
    print(f"[INFO]: saving models.")
if run.job_type == "debug":
    log = "all"

In [ ]:
if config["log_wandb"]:
    wandb_logger = pl.loggers.WandbLogger()
    wandb_logger.watch(model, log=log, log_graph=True)
else:
    wandb_logger = None
trainer = pl.Trainer(
    max_epochs=150,
    callbacks=callbacks,
    logger=wandb_logger,
    devices="auto", 
    accelerator="auto",
#     limit_train_batches=0.3, 
#     limit_val_batches=0.3,
#     log_every_n_steps=1
)
trainer.fit(model, datamodule=dm, )

In [ ]:
dm.prepare_data()
dm.setup("validate")
predictions = torch.cat(trainer.predict(model, dataloaders=dm.predict_dataloader()))
targets = torch.FloatTensor(dm.targets)

In [ ]:
save_output = training.SaveOutput()
handle = model.conv_layers[3].register_forward_hook(save_output)
image = images[0]
model(image.unsqueeze(0))
handle.remove()
outputs = save_output.outputs[0].permute(1, 0, 2, 3).detach().cpu()[:30]
grid_img = utils.make_grid(outputs, normalize=True, pad_value=1, padding=1)
plt.figure(figsize=(30, 30))
plt.imshow(grid_img.permute(1, 2, 0))

In [ ]:
import seaborn as sns

fig, ax = plt.subplots()
sns.kdeplot(
    data=dm.scaler.transform(targets.cpu().numpy())[:, 4],
    color="orange",
    ax=ax,
    label="target",
)
sns.kdeplot(
    data=predictions.cpu().numpy(),
    shade=True,
    color="orange",
    ax=ax,
    label="predictions",
)
ax.legend()

In [ ]:
run.finish()

In [ ]:
for noise in [0.9, 0.3, 0]:
    for layer_divider in [4, 16]:
        for fc_divider in [4, 16]:
            for dropout in [0.2, 0.6]:
                if log_wandb:
                    import wandb
                    wandb_api.login()
                    run = wandb.init(
                        project="microstructure-reconstruction",
                        group="Naive Network",
                        job_type="test",
                    )

                if log_wandb:
                    config = wandb.config
                else:
                    config = wandb_api.Config()

                config["job_type"] = run.job_type
                config["train_val_split"] = 0.7
                config["seed"] = 42
                config["batch_size"] = 32
                config["learning_rate"] = 0.00006
                config["device"] = device
                config["momentum"] = 0.9
                config["architecture"] = "VGG"
                config["input_width"] = 64
                config["weight_decay"] = 0.005
                config["epochs"] = 0
                config["frac_sample"] = 1
                config["frac_noise"] = noise
                # config["total_layers"] = 24
                # config["fixed_layers"] = 0
                config["log_wandb"] = log_wandb
                torch.manual_seed(config["seed"])
                pl.seed_everything(config["seed"])

                class VGG11(models.BaseModel):
                    def __init__(self, config, kde = None, scaler=None):
                        super().__init__(config)

                        self.config = config
                        self.config["model_type"] = type(self)
                        self.kde = kde
                        self.scaler = scaler

                        self.configure_model()
                        self.configure_criterion()
                        self.configure_metrics()

                    def configure_model(self):
                        layers = np.array([64, 128, 256, 512]) // layer_divider
                        self.conv_layers = nn.Sequential(
                            nn.Conv2d(1, layers[0], kernel_size=3, padding=1),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=2, stride=2),
                            nn.Conv2d(layers[0], layers[1], kernel_size=3, padding=1),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=2, stride=2),
                            nn.Conv2d(layers[1], layers[2], kernel_size=3, padding=1),
                            nn.ReLU(),
                            nn.Conv2d(layers[2], layers[2], kernel_size=3, padding=1),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=2, stride=2),
                            nn.Conv2d(layers[2], layers[3], kernel_size=3, padding=1),
                            nn.ReLU(),
                            nn.Conv2d(layers[3], layers[3], kernel_size=3, padding=1),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=2, stride=2),
                            nn.Conv2d(layers[3], layers[3], kernel_size=3, padding=1),
                            nn.ReLU(),
                            nn.Conv2d(layers[3], layers[3], kernel_size=3, padding=1),
                            nn.BatchNorm2d(layers[3]),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=2, stride=2)
                #             nn.MaxPool2d(kernel_size=int(self.config["input_width"] / (2 ** 4)), stride=int(self.config["input_width"] / (2 ** 4))),
                        )
                        input_fc = int((self.config["input_width"] / (2 ** 5)) ** 2 * layers[3])
                        # fully connected linear layers
                        n = 256 // fc_divider
                        self.linear_layers = nn.Sequential(
                            nn.Flatten(),
                            nn.Linear(in_features=input_fc, out_features=n),
                            nn.ReLU(),
                            nn.Dropout2d(dropout),
                            nn.Linear(in_features=n, out_features=1),
                #             nn.ReLU(),
                #             nn.Dropout2d(0.3),
                #             nn.Linear(in_features=n, out_features=1),
                        )

                    def forward(self, x):
                        x = self.conv_layers(x)
                        # flatten to prepare for the fully connected layers
                #         x = x.view(x.size(0), -1)
                        x = self.linear_layers(x)
                        return x

                    def training_step(self, batch, batch_idx):
                        x, y = batch
                        y_hat = self(x)
                #         weights = self.kde.score_samples(y_hat.cpu().detach().numpy())
                        loss = self.criterion(y_hat, y)
                        self.log(
                            "train_loss",
                            loss,
                            on_step=False,
                            on_epoch=True,
                        )
                        return loss

                    def validation_step(self, batch, batch_idx):
                        x, y = batch
                        y_hat = self(x)
                #         weights = self.kde.score_samples(y_hat.cpu().detach().numpy())
                        metrics = {name: metric(y, y_hat) for name, metric in self.metrics.items()}
                        metrics["val_loss"] = self.criterion(y_hat, y)
                        self.log_dict(metrics, on_step=False, on_epoch=True, prog_bar=True)
                        return metrics

                    def configure_criterion(self):
                        self.criterion = self.L1Loss
                        self.config["loss_type"] = type(self.criterion)

                    def L1Loss(self, y, y_hat, weights=None):
                        if weights is not None:
                            weights = nn.functional.normalize(weights, p=1, dim=0)
                            return torch.mean(torch.matmul(weights, torch.abs(y - y_hat)))
                        else: 
                            return torch.mean(torch.abs(y - y_hat))

                    def configure_metrics(self):
                        self.metrics = {
                            "mae": torchmetrics.MeanAbsoluteError().to(self.config["device"]),
                            "mape": torchmetrics.MeanAbsolutePercentageError().to(
                                self.config["device"]
                            ),
                            "smape": torchmetrics.SymmetricMeanAbsolutePercentageError().to(
                                self.config["device"]
                            ),
                            "r2_score": torchmetrics.R2Score(num_outputs=23).to(self.config["device"]),
                            "cosine_similarity": torchmetrics.CosineSimilarity(reduction="mean").to(
                                self.config["device"]
                            ),
                        }

                    def configure_optimizers(self):
                        optimizer = torch.optim.Adam(
                            self.parameters(),
                            lr=self.config["learning_rate"],
                            weight_decay=self.config["weight_decay"],
                        )
                        self.config["optimizer_type"] = type(optimizer)
                        return optimizer

                dm = DataModule(config, repo_path)

                model = VGG11(config, kde=None)
                total_params = sum(p.numel() for p in model.parameters())
                print(f"NOISE {noise}\tFC DIVIDER {fc_divider}\tLAYER DIVIDER {layer_divider}\tDROPOUT {dropout}")
                print(f"[INFO]: {total_params:,} total parameters.")
                model(torch.rand((2, 1, config["input_width"], config["input_width"])))

                model_checkpoint = pl.callbacks.model_checkpoint.ModelCheckpoint(
                    dirpath=run.dir,
                    filename="{epoch}-{val_loss:.3f}",
                    monitor="val_loss",
                    mode="min",
                    verbose=True,
                    save_last=True,
                )

    #             script_checkpoint = training.ScriptCheckpoint(
    #                 dirpath=run.dir,
    #             )

    #             callbacks = [script_checkpoint]
                callbacks = []
                log = None
                if run.job_type == "train" or True:
                    callbacks.append(model_checkpoint)
                    print(f"[INFO]: saving models.")
                if run.job_type == "debug":
                    log = "all"

                if config["log_wandb"]:
                    wandb_logger = pl.loggers.WandbLogger()
                    wandb_logger.watch(model, log=log, log_graph=True)
                else:
                    wandb_logger = None
                trainer = pl.Trainer(
                    max_epochs=30,
                    callbacks=callbacks,
                    logger=wandb_logger,
                    devices="auto", 
                    accelerator="auto",
                #     limit_train_batches=0.3, 
                #     limit_val_batches=0.3,
                #     log_every_n_steps=1
                )
                trainer.fit(model, datamodule=dm, )
